In [1]:
import torch
import torch.nn as nn
import torch.utils.data as Data
import torchvision
import numpy as np
import sys
sys.path.append("..")
import d2lzh_pytorch as d2l

In [2]:
num_inputs, num_outputs, num_hiddens = 28*28, 10, 256

In [8]:
net = nn.Sequential(
    d2l.FlattenLayer(),
    nn.Linear(num_inputs, num_hiddens),
    nn.ReLU(),
    nn.Linear(num_hiddens, num_outputs),
)
for params in net.parameters():
    nn.init.normal_(params, mean=0, std=0.01)

if torch.cuda.is_available():
    print("cuda!")
    net = nn.DataParallel(net, device_ids=[0])
print(net)

cuda!
DataParallel(
  (module): Sequential(
    (0): FlattenLayer()
    (1): Linear(in_features=784, out_features=256, bias=True)
    (2): ReLU()
    (3): Linear(in_features=256, out_features=10, bias=True)
  )
)


In [10]:
batch_size = 256
train_data = torchvision.datasets.FashionMNIST(
    root='../Datasets/FashionMNIST/',
    train = True,
    download = False,
    transform = torchvision.transforms.ToTensor(),
)
train_iter = Data.DataLoader(dataset=train_data, batch_size=batch_size, shuffle=True)

In [13]:
optimizer = torch.optim.SGD(net.parameters(), lr=0.5)
loss_func = nn.CrossEntropyLoss().cuda()
num_epochs = 5

for epoch in range(num_epochs):
    for X, y in train_iter:
        b_x = X.cuda(); b_y = y.cuda()
        output = net(b_x)
        loss = loss_func(output, b_y)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
    
    print( "epoch: %d, loss: %.4f" % (epoch, loss.data.cpu().numpy()) )

epoch: 0, loss: 0.6387
epoch: 1, loss: 0.4726
epoch: 2, loss: 0.3171
epoch: 3, loss: 0.2717
epoch: 4, loss: 0.5056
